### 5. EDA & 모델링(2)
- 데이터 시각화
- 피처간 연관성 찾기

### 1. 데이터 읽어오기

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')      
submission = pd.read_csv('sample_submission.csv')

train.head(3)

### 2. 데이터 확인

In [ ]:
id = 405136001
number = '360-1'

select_nm = train[train['route_id'] == id]['route_nm'].unique()
select_id = train[train['route_nm'] == number]['route_id'].unique()

print(select_nm)
print(select_id)

In [ ]:
print(train[train['route_id'] == 405136001]['vh_id'].unique())
print(train[train['route_nm'] == '360-1']['vh_id'].unique())

In [ ]:
print(train[train['vh_id'] == 7997025]['route_id'].unique())
print(train[train['vh_id'] == 7997025]['route_nm'].unique())

In [ ]:
def check_unique(text:str):
    if (len(train[train['vh_id'] == text]['route_id'].unique()) != 1) |    \
        (len(train[train['vh_id'] == text]['route_nm'].unique()) != 1):
        return True
    else :
        return False

check_vhid = list(map(check_unique, train['vh_id'].unique()))
set(check_vhid)

In [ ]:
train.loc[178:183][['vh_id']]

In [ ]:
train['shifted_vhid'] = train['vh_id'].shift(-1)
train.loc[178:183][['vh_id', 'shifted_vhid']]

In [ ]:
idx = [-1]     
idx = idx + list(train[train['vh_id'] != train['shifted_vhid']]['id'].values)
idx[:10]

In [ ]:
import folium

# 0~1329 사이의 값을 입력하여 경로를 확인해보세요!
my_idx = 700

long = train['now_longitude'].mean()
lat = train['now_latitude'].mean()
center = [lat,long]

m = folium.Map(location=center, zoom_start=10)

# my_idx가 700일 경우 train.loc[114015:114139][['next_latitude', 'next_longitude']].values와 동일
locations = train.loc[idx[my_idx]+1:idx[my_idx+1]][['next_latitude', 'next_longitude']].values

folium.PolyLine(locations=locations).add_to(m)

m

In [ ]:
pd.DataFrame(train.groupby(['now_station', 'now_latitude']).count().reset_index()[:50].value_counts('now_station'))

In [ ]:
def check_station(text:str):
    if len(train[train['now_latitude'] == text]['now_station'].unique()) != 1:
        return True
    else :
        return False

check_nowlat = list(map(check_station, train['now_latitude'].unique()))
set(check_nowlat)

In [ ]:
def check_station(text:str):
    if len(train[train['now_longitude'] == text]['now_station'].unique()) != 1:
        return True
    else :
        return False

check_nowlong = list(map(check_station, train['now_longitude'].unique()))
set(check_nowlong)

In [ ]:
def check_station(text:str):
    if len(train[train['next_longitude'] == text]['next_station'].unique()) != 1:
        return True
    else :
        return False

check_next_long = list(map(check_station, train['next_longitude'].unique()))
set(check_next_long)

In [ ]:
# 파이썬 warning 무시
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트를 사용하기 위한 코드
fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

## 코드 작성
x = ['911의원1', '911의원2']
y = [train[train['now_latitude'] == train[train['now_station'] == '911의원']['now_latitude'].unique()[0]]['next_arrive_time'].mean(),
     train[train['now_latitude'] == train[train['now_station'] == '911의원']['now_latitude'].unique()[1]]['next_arrive_time'].mean()]

plt.figure(dpi=150)

plt.title('정류장 별 운행시간')
plt.xlabel('정류장')
plt.ylabel('운행시간')

plt.bar(x,y)
plt.show()

In [ ]:
x = ['고성리제주은행1', '고성리제주은행2']
y = [train[train['now_latitude'] == train[train['now_station'] == '고성리제주은행']['now_latitude'].unique()[0]]['next_arrive_time'].mean(),
     train[train['now_latitude'] == train[train['now_station'] == '고성리제주은행']['now_latitude'].unique()[1]]['next_arrive_time'].mean()]

plt.figure(dpi=150)

plt.title('정류장 별 운행시간')
plt.xlabel('정류장')
plt.ylabel('운행시간')

plt.bar(x,y)
plt.show()

In [ ]:
train['now_arrive_time'].value_counts().sort_index()

In [ ]:
x = train['now_arrive_time'].value_counts().sort_index().index
y = train['now_arrive_time'].value_counts().sort_index().values

# fig와 ax 객체 생성
fig, ax = plt.subplots(dpi=150)

ax.bar(x, y)

# x축의 라벨값을 45도로 기울임
plt.xticks(rotation=45)

plt.show()

In [ ]:
ax = train['distance'].plot(figsize=(20,10))
plt.ylabel('distance')
plt.show()

In [ ]:
train[train['distance'] > 6000]['vh_id'].unique()

In [ ]:
bus_id = 7983416
date = '2019-10-16'

locations = train[(train['vh_id'] == bus_id) & (train['date'] == date)][['now_latitude', 'now_longitude']].values
lat = locations[:,0].mean()
long = locations[:,1].mean()
center = [lat,long]

m = folium.Map(location=center, zoom_start=10)

folium.PolyLine(locations=locations).add_to(m)

m

### 3. 데이터 전처리

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
my_list = list(train['vh_id'].unique()) + list(train['now_longitude'].unique()) + list(train['next_longitude'].unique())   
my_dict = {text : i for i, text in enumerate(my_list)}

In [ ]:
train = train[train['next_arrive_time'] < 1000]

In [ ]:
features = ['vh_id', 'now_longitude', 'now_arrive_time', 'distance', 'next_longitude']

# 학습에 사용할 변수 X 할당
X = train[features]

# 예측할 변수 y 할당
y = train['next_arrive_time']

### 4. 데이터 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 모델 선언
model = RandomForestRegressor(n_estimators=20)

# 모델 학습
model.fit(X, y)

# 예측값 획득
# features = ['vh_id', 'now_longitude', 'now_arrive_time', 'distance', 'next_longitude']
predict = model.predict(test[features])

### 5. CSV파일로 저장

In [ ]:
submission['next_arrive_time'] = predict
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False, quoting=2, encoding="utf-8-sig", na_rep=0, float_format='%.6f')